## Import Dependencies

In [ ]:
# List of external libraries used:
# pandas: https://pandas.pydata.org
# numpy: https://numpy.org
# matplotlub: https://matplotlib.org
# sklearn: https://scikit-learn.org

In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir("drive/My Drive/Colab Notebooks/CMPT419Project/model")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from sklearn.utils import shuffle

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AvgPool2D, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

2022-03-24 15:50:31.385045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hvtruong/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-24 15:50:31.385121: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Preprocessing:

## Landmarks detection

In [2]:
import dlib

predictor = dlib.shape_predictor('../Dataset/utils/shape_predictor_68_face_landmarks.dat')

def get_landmarks(face):
    landmarks = predictor(face, dlib.rectangle(0,0,63,63))
    landmarks_image = np.zeros((64,64))
    for i in range(0, 68):
        cv2.circle(landmarks_image, (landmarks.part(i).x, landmarks.part(i).y), 1, 255, 1)
  
    return landmarks_image

## Load the dataset

In [3]:
import h5py

filename = "../Dataset/facial_data/facial_data.h5"

data = np.zeros((25844,64,64,2))

with h5py.File(filename, "r") as f:
    keys = list(f.keys())
    # Get the data
    for i in range(int(len(keys))):
      data[i,:,:,0] = (np.array(f[keys[i]]))
      data[i,:,:,1] = (get_landmarks(np.array(f[keys[i]])))

data = np.array(data)

df = pd.read_csv('../Dataset/src_data/label_data.csv')
df = df[df['file_name'].isin(keys)]
targets = df[['arousal', 'valence']].values

data, targets = shuffle(data, targets)

# Model

## Model take input as gray images

In [ ]:
learning_rate = 0.0001
epochs = 30
batch_size = 64

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='../models/grayImagesModel.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
kf = KFold(n_splits=5)

loss = []
val_loss = []

for train_index, val_index in kf.split(data):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=(3,3), activation='relu',input_shape=(64,64,1), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(16, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())

    #model.add(MaxPool2D(strides=(2,2)))
    model.add(AvgPool2D(strides=(2,2)))
    model.add(Dropout(0.25))
    #Output dimension: 32x32x16

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())

    #model.add(MaxPool2D(strides=(2,2)))
    model.add(AvgPool2D(strides=(2,2)))
    model.add(Dropout(0.25))
    #Output dimension: 16x16x32

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.4))

    model.add(Dense(2, activation='linear'))

    model.compile(loss = 'mse', optimizer = Adam(learning_rate))

    x_train = data[train_index]
    y_train = targets[train_index]
    x_val = data[val_index]
    y_val = targets[val_index]

    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[callbacks], verbose=1, validation_data= (x_val, y_val))

    loss.append(history.history['loss'][-1])
    val_loss.append(history.history['val_loss'][-1])

model.save('./grayImagesModel.h5')

In [ ]:
print(np.mean(loss))
print(np.mean(val_loss))

In [ ]:
plt.figure(figsize=(6, 5))
# training loss
plt.plot(history.history['loss'], color='r')
#validation loss
plt.plot(history.history['val_loss'], color='g')
plt.show()

In [ ]:
y_pred = model.predict(x_val)
for rand_num in np.random.randint(0, len(y_val), 10):
    plt.figure()
    plt.imshow(x_val[rand_num].reshape(48, 48),cmap='gray')
    plt.axis('off')
    plt.title('Prediction: ' + str(y_pred[rand_num]) + ' Real: ' + str(y_val[rand_num]), color='g')

## Model takes inputs as gray images with an addtional landmarks layer

In [15]:
learning_rate = 0.0001
epochs = 30
batch_size = 64

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='../models/2layersInputModel.h5', monitor='val_loss', save_best_only=True)]

In [16]:
kf = KFold(n_splits=5)

loss = []
val_loss = []

for train_index, val_index in kf.split(data):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=(3,3), activation='relu',input_shape=(64,64,2), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(16, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())

    #model.add(MaxPool2D(strides=(2,2)))
    model.add(AvgPool2D(strides=(2,2)))
    model.add(Dropout(0.25))
    #Output dimension: 128x128x16

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())

    #model.add(MaxPool2D(strides=(2,2)))
    model.add(AvgPool2D(strides=(2,2)))
    model.add(Dropout(0.25))
    #Output dimension: 64x64x32

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.4))

    model.add(Dense(2, activation='linear'))

    model.compile(loss = 'mse', optimizer = Adam(learning_rate))

    x_train = data[train_index]
    y_train = targets[train_index]
    x_val = data[val_index]
    y_val = targets[val_index]

    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[callbacks], verbose=1, validation_data= (x_val, y_val))

    loss.append(history.history['loss'][-1])
    val_loss.append(history.history['val_loss'][-1])

2022-03-24 09:51:28.395375: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 677478400 exceeds 10% of free system memory.


Epoch 1/30
324/324 [==============================] - ETA: 0s - loss: 6.6782

2022-03-24 09:54:11.606623: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 169377792 exceeds 10% of free system memory.


324/324 [==============================] - 169s 513ms/step - loss: 6.6782 - val_loss: 5.5290
Epoch 2/30
324/324 [==============================] - 155s 479ms/step - loss: 5.3150 - val_loss: 4.1967
Epoch 3/30
324/324 [==============================] - 156s 482ms/step - loss: 4.3613 - val_loss: 3.5326
Epoch 4/30
324/324 [==============================] - 154s 476ms/step - loss: 3.7010 - val_loss: 3.1867
Epoch 5/30
324/324 [==============================] - 153s 472ms/step - loss: 3.2701 - val_loss: 2.8422
Epoch 6/30
324/324 [==============================] - 151s 466ms/step - loss: 2.9136 - val_loss: 2.6661
Epoch 7/30
324/324 [==============================] - 136s 420ms/step - loss: 2.6933 - val_loss: 2.4939
Epoch 8/30
324/324 [==============================] - 136s 419ms/step - loss: 2.4867 - val_loss: 2.3353
Epoch 9/30
324/324 [==============================] - 137s 422ms/step - loss: 2.2398 - val_loss: 2.2458
Epoch 10/30
324/324 [==============================] - 136s 421ms/step - lo

2022-03-24 10:49:12.862346: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 677478400 exceeds 10% of free system memory.


Epoch 1/30
324/324 [==============================] - ETA: 0s - loss: 6.5295

2022-03-24 10:51:46.538131: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 169377792 exceeds 10% of free system memory.


324/324 [==============================] - 160s 489ms/step - loss: 6.5295 - val_loss: 5.6492
Epoch 2/30
324/324 [==============================] - 151s 465ms/step - loss: 5.1363 - val_loss: 4.4548
Epoch 3/30
324/324 [==============================] - 148s 457ms/step - loss: 4.1802 - val_loss: 3.4390
Epoch 4/30
324/324 [==============================] - 149s 461ms/step - loss: 3.5558 - val_loss: 3.1393
Epoch 5/30
324/324 [==============================] - 149s 460ms/step - loss: 3.1612 - val_loss: 2.7888
Epoch 6/30
324/324 [==============================] - 149s 459ms/step - loss: 2.8491 - val_loss: 2.6720
Epoch 7/30
324/324 [==============================] - 148s 458ms/step - loss: 2.5730 - val_loss: 2.4958
Epoch 8/30
324/324 [==============================] - 141s 436ms/step - loss: 2.4381 - val_loss: 2.4186
Epoch 9/30
324/324 [==============================] - 134s 414ms/step - loss: 2.2325 - val_loss: 2.2758
Epoch 10/30
324/324 [==============================] - 135s 417ms/step - lo

2022-03-24 11:47:32.958777: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 677478400 exceeds 10% of free system memory.


Epoch 1/30
324/324 [==============================] - 137s 419ms/step - loss: 6.6398 - val_loss: 5.4804
Epoch 2/30
324/324 [==============================] - 135s 416ms/step - loss: 5.3207 - val_loss: 4.3035
Epoch 3/30
324/324 [==============================] - 134s 415ms/step - loss: 4.3062 - val_loss: 3.5966
Epoch 4/30
324/324 [==============================] - 134s 413ms/step - loss: 3.7149 - val_loss: 3.2500
Epoch 5/30
324/324 [==============================] - 135s 418ms/step - loss: 3.2756 - val_loss: 3.0410
Epoch 6/30
324/324 [==============================] - 134s 414ms/step - loss: 2.9554 - val_loss: 2.8521
Epoch 7/30
324/324 [==============================] - 134s 415ms/step - loss: 2.7016 - val_loss: 2.6310
Epoch 8/30
324/324 [==============================] - 135s 417ms/step - loss: 2.5100 - val_loss: 2.5397
Epoch 9/30
324/324 [==============================] - 136s 419ms/step - loss: 2.3343 - val_loss: 2.4633
Epoch 10/30
324/324 [==============================] - 135s 418m

In [17]:
print(np.mean(loss))
print(np.mean(val_loss))

1.132283353805542
1.8107167959213257


In [ ]:
plt.figure(figsize=(6, 5))
# training loss
plt.plot(history.history['loss'], color='r')
#validation loss
plt.plot(history.history['val_loss'], color='g')
plt.show()

In [ ]:
y_pred = model.predict(x_val)
for rand_num in np.random.randint(0, len(y_val), 10):
    plt.figure()
    plt.imshow(x_val[rand_num].reshape(48, 48),cmap='gray')
    plt.axis('off')
    plt.title('Prediction: ' + str(y_pred[rand_num]) + ' Real: ' + str(y_val[rand_num]), color='g')

## Model takes inputs as gray images with landmarks concatenated at latent space

In [8]:
learning_rate = 0.001
epochs = 30
batch_size = 64

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='../models/landmarksConcatenatedModel.h5', monitor='val_loss', save_best_only=True)]

In [9]:
kf = KFold(n_splits=5)

loss = []
val_loss = []

for train_index, val_index in kf.split(data):

    img_input = Input((64,64,1))
    landmarks_input = Input((64*64))

    cnnLayers = Conv2D(16, kernel_size=(3,3), activation='relu',input_shape=(64,64,1), padding='same')(img_input)
    cnnLayers = BatchNormalization()(cnnLayers)
    cnnLayers = Conv2D(16, kernel_size=(3,3), activation='relu', padding='same')(cnnLayers)
    cnnLayers = BatchNormalization()(cnnLayers)

    cnnLayers = AvgPool2D(strides=(2,2))(cnnLayers)
    cnnLayers = Dropout(0.25)(cnnLayers)

    #model.add(MaxPool2D(strides=(2,2)))
    #Output dimension: 128x128x16

    cnnLayers = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(cnnLayers)
    cnnLayers = BatchNormalization()(cnnLayers)
    cnnLayers = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(cnnLayers)
    cnnLayers = BatchNormalization()(cnnLayers)

    cnnLayers = AvgPool2D(strides=(2,2))(cnnLayers)
    cnnLayers = Dropout(0.25)(cnnLayers)

    cnnLayers = AvgPool2D(strides=(2,2))(cnnLayers)
    cnnLayers = Dropout(0.25)(cnnLayers)

    #model.add(MaxPool2D(strides=(2,2)))
    #Output dimension: 64x64x32

    flatten_layer = Flatten()(cnnLayers)
    
    concatenate_layer = concatenate([flatten_layer, landmarks_input])
    dense = Dense(512, activation='relu')(concatenate_layer)
    dense = Dropout(0.25)(dense)

    dense = Dense(1024, activation='relu')(dense)
    dense = Dropout(0.4)(dense)

    output = Dense(2, activation='linear')(dense)

    model = Model(inputs=[img_input, landmarks_input], outputs=output)
    model.compile(loss = 'mse', optimizer = Adam(learning_rate))

    x_train = data[train_index]
    y_train = targets[train_index]
    x_val = data[val_index]
    y_val = targets[val_index]

    history = model.fit(x=[x_train[:,:,:,0], np.reshape(x_train[:,:,:,1],((x_train.shape[0],64*64)))], y=y_train, epochs=epochs, batch_size=batch_size, callbacks=[callbacks], verbose=1, validation_data= ([x_val[:,:,:,0], np.reshape(x_val[:,:,:,1],((x_val.shape[0],64*64)))], y_val))

    loss.append(history.history['loss'][-1])
    val_loss.append(history.history['val_loss'][-1])

Epoch 1/30
102/324 [========>.....................] - ETA: 2:13 - loss: 914.5102

In [ ]:
print(np.mean(loss))
print(np.mean(val_loss))

plt.figure(figsize=(6, 5))
# training loss
plt.plot(history.history['loss'], color='r')
#validation loss
plt.plot(history.history['val_loss'], color='g')
plt.show()

In [ ]:
y_pred = model.predict(x_val)
for rand_num in np.random.randint(0, len(y_val), 10):
    plt.figure()
    plt.imshow(x_val[rand_num].reshape(48, 48),cmap='gray')
    plt.axis('off')
    plt.title('Prediction: ' + str(y_pred[rand_num]) + ' Real: ' + str(y_val[rand_num]), color='g')